In [222]:
# Import third party libraries

# Numerical library
import numpy as np

# Machine learning library
from sklearn import linear_model
from sklearn import metrics

# Used for matrix inversion
from numpy.linalg import inv

# Plotting library
import matplotlib.pyplot as plt

# Description of each column
desc = {
    0:'CRIM', 1:'ZN', 2:'INDUS', 
    3:'CHAS', 4:'NOX', 5:'RM', 
    6:'AGE', 7:'DIS', 8:'RAD', 
    9:'TAX', 10:'PTRATIO', 11:'B', 
    12:'LSTAT', 13:'MEDV'
}

# Allows for printing inline for jupyter notebook
%matplotlib inline 

{0: 'CRIM', 1: 'ZN', 2: 'INDUS', 3: 'CHAS', 4: 'NOX', 5: 'RM', 6: 'AGE', 7: 'DIS', 8: 'RAD', 9: 'TAX', 10: 'PTRATIO', 11: 'B', 12: 'LSTAT', 13: 'MEDV'}


In [140]:
# Load datasets and store in ndarray
training_data = open('housing_train.txt','r')
data_train = np.loadtxt(training_data)

testing_data = open('housing_test.txt', 'r')
data_test = np.loadtxt(testing_data)

In [141]:
# Split off known target values
y_init_train = data_train[:,13]
y_init_test = data_test[:,13]

# Transpose row vector to columnar
y_train = y_init_train[np.newaxis].T
y_test = y_init_test[np.newaxis].T

In [142]:
# Remove column 13 from X
X_init_train = np.delete(data_train, 13, axis=1)
X_init_test = np.delete(data_test, 13, axis=1)

# Function to create array of dummy ones and returned 
# columnar vector
def make_dummy_vector(target):
    temp = np.ones(len(target))
    return temp[np.newaxis].T

# Create dummy 1 values
dummy_train = make_dummy_vector(X_init_train)
dummy_test = make_dummy_vector(X_init_test)

# Add dummy data to feature matrices
X_train = np.concatenate((dummy_train, X_init_train), axis=1)
X_test = np.concatenate((dummy_test, X_init_test), axis=1)

In [143]:
## PART 2
# Compute optimal weight vector w -- (X^T * X)^-1 (X^T * Y)
def calc_w_vector(X, y):
    return np.dot(inv(np.dot(X.T,X)), np.dot(X.T,y))
    
def alternate_calc(X, y):
    return np.dot(np.dot(inv(X),inv(X.T)), np.dot(X.T,y))

# Limit printout to 3 decimal places
np.set_printoptions(precision=3)

# Caculate w vectors
w_train = calc_w_vector(X_train,y_train)
w_test = calc_w_vector(X_test,y_test)

# Print both weight vectors to console
print 'w_train vector:'
print('\n'.join('{}: {}'.format(*k) for k in enumerate(w_train)))

print ' \r\nw_test vector:'
print('\n'.join('{}: {}'.format(*k) for k in enumerate(w_test)))

w_train vector:
0: [ 39.584]
1: [-0.101]
2: [ 0.046]
3: [-0.003]
4: [ 3.072]
5: [-17.225]
6: [ 3.711]
7: [ 0.007]
8: [-1.599]
9: [ 0.374]
10: [-0.016]
11: [-1.024]
12: [ 0.01]
13: [-0.586]
 
w_test vector:
0: [ 16.494]
1: [-0.03]
2: [ 0.01]
3: [-0.16]
4: [ 1.129]
5: [-6.583]
6: [ 4.438]
7: [-0.077]
8: [-0.845]
9: [-0.025]
10: [ 0.005]
11: [-0.7]
12: [ 0.01]
13: [-0.037]


In [231]:
## PART 3
# Functions
# def calc_sse(X, y, regr):
#     return np.sum((regr.predict(X) - y) ** 2)

# def calc_mse(X, y, regr):
#     return np.mean((regr.predict(X) - y) ** 2)

def format_data(array):
    result, i = {}, 0
    while i < len(array):
        j = 0;
        while j < len(array[i]):
            result.setdefault(j, [])
            result[j].append(array[i][j])
            j += 1
        i += 1
    return result.values()

def calc_b(X_mean, y_mean, w):
    bs, i = [], 0
    while i < len(w) - 1:
        bs.append(y_mean - w[i] * X_mean[i])
        i += 1
    return bs
    
def calc_mean(array):
    means, i = [], 0
    while i < len(array):
        means.append(np.mean(array[i]))
        i += 1
    return means

def calc_sqrt_mean(array):
    means, i = [], 0
    while i < len(array):
        means.append(np.mean([item * item for item in array[i]]))
        i += 1
    return means

def calc_sse(X, y, w, b):
    sse, i = [], 0
    while i < len(X):
        sum, length = 0, len(X[i])
        for j in range(length):
            sum += y[i] - w[i] * X[i][j] - b[i]
        sse.append(sum[0])
        i += 1
    return sse

X_format_train = format_data(X_init_train)
X_train_mean = calc_mean(X_format_train)
y_train_mean = np.mean(y_init_train)
b_train = calc_b(X_train_mean, y_train_mean, w_train)
sse = calc_sse(X_format_train, y_init_train, w_train, b_train)
# Print error output, not sure about the 0 values
print 'Training Model SSE:'
for i in range(len(sse)):
    print desc[i] + '-[', sse[i], ']'
    
print

X_format_test = format_data(X_init_test)
X_test_mean = calc_mean(X_format_test)
y_test_mean = np.mean(y_init_test)
b_test = calc_b(X_test_mean, y_test_mean, w_test)
sse = calc_sse(X_format_test, y_init_test, w_test, b_test)
# Print error output, not sure about the 0 values
print 'Testing Model SSE:'
for i in range(len(sse)):
    print desc[i] + '-[', sse[i], ']'
    
# Apply learned weight vectors
# print 'MSE: %.2f \r\n' % calc_mse(X_train, target_func_train, testing_model)
# target_func_train = np.dot(X_train, w_train)
# target_func_test = np.dot(X_test, w_test)

# Create linear regression object
# training_model = linear_model.LinearRegression()
# testing_model = linear_model.LinearRegression()

# Train the model using the training sets
# training_model.fit(X_train,target_func_train)
# testing_model.fit(X_test,target_func_test)

# Print error output, not sure about the 0 values
# print 'Training Model: \r\nSSE: %.2f' % calc_sse(X_train, target_func_train, training_model)
# print 'MSE: %.2f \r\n' % calc_mse(X_train, target_func_train, testing_model)

# print 'Testing Model: \r\nSSE: %.2f' % calc_sse(X_test, target_func_test, training_model)
# print 'MSE: %.2f' % calc_mse(X_test, target_func_test, testing_model)

#metrics.mean_squared_error(y_train,target_func_train)
#metrics.mean_squared_error(y_test,target_func_test)


ValueError: shapes (433,13) and (14,1) not aligned: 13 (dim 1) != 14 (dim 0)

In [ ]:
## ATTEMPT TO EXPRESS THE OPTIMIZED SSE FORMULA
print np.sum(np.dot(np.subtract(target_func_train_no_dummy.T, np.dot(X_train_no_dummy, w_train_no_dummy).T), np.subtract(target_func_train_no_dummy.T, np.dot(X_train_no_dummy, w_train_no_dummy))))

In [ ]:
## PART 4
# Repeating part 2 and 3 without a dummy features of 1's in X

# Remove dummy column from both tables
X_train_no_dummy = X_train[:, (1,2,3,4,5,6,7,8,9,10,11,12,13)]
X_test_no_dummy = X_test[:, (1,2,3,4,5,6,7,8,9,10,11,12,13)]

# Caculate w vectors
w_train_no_dummy = calc_w_vector(X_train_no_dummy,y_train)
w_test_no_dummy = calc_w_vector(X_test_no_dummy,y_test)

# Print both weight vectors to console
print 'w_train_no_dummy vector:'
print('\n'.join('{}: {}'.format(*k) for k in enumerate(w_train_no_dummy)))

print ' \r\nw_test_no_dummy vector:'
print('\n'.join('{}: {}'.format(*k) for k in enumerate(w_test_no_dummy)))

<h3>Thoughts about results</h3>
The above results make it seems like we our model will be centered around the orgin beacuse we did not calcuate a true b value in the w vector.

In [ ]:
## PART 4 cont.
# Apply learned weight vectors
target_func_train_no_dummy = np.dot(X_train_no_dummy, w_train_no_dummy)
target_func_test_no_dummy = np.dot(X_test_no_dummy, w_test_no_dummy)

# Create linear regression object
training_model_no_dummy = linear_model.LinearRegression()
testing_model_no_dummy = linear_model.LinearRegression()

# Train the model using the training sets
training_model_no_dummy.fit(X_train_no_dummy,target_func_train_no_dummy)
testing_model_no_dummy.fit(X_test_no_dummy,target_func_test_no_dummy)

# Print error output, not sure about the 0 values
print 'Training Model without Dummy: \r\nSSE: %.2f' % calc_sse(X_train_no_dummy, target_func_train_no_dummy, training_model_no_dummy)
print 'MSE: %.2f \r\n' % calc_mse(X_train_no_dummy, target_func_train, training_model_no_dummy)

print 'Testing Model without dummy: \r\nSSE: %.2f' % calc_sse(X_test_no_dummy, target_func_test_no_dummy, testing_model_no_dummy)
print 'MSE: %.2f' % calc_mse(X_test_no_dummy, target_func_test_no_dummy, training_model_no_dummy)

In [ ]:
# Coefficients
print('Coefficients: \n' , regr.coef_)

# Mean squared error
print("Mean squared error: %.2f" 
      % np.mean((regr.predict(X_test) - y_test) ** 2))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(X_test, y_test))

In [ ]:
print (regr.predict(X_test[:1,]) - y_test[:1,])

In [ ]:
print X_train[:,2]

In [ ]:
# EXTRA MATERIAL BELOW

In [ ]:
# Don't show scientific notation
np.set_printoptions(suppress=True)

print "Printing X_train:"
print X_train

In [ ]:
print "Printing y_train:"
print y_train

In [ ]:
# Plot feature 1: Crime rate by town
plt.scatter(X_train[:, 0],y_train)

In [ ]:
# Plot feature 2: Residential land zoned for lots over 25,0000 sq. ft
plt.scatter(X_train[:, 1],y_train)

In [ ]:
# Multiplotting feature 1 & 2
plt.scatter(X_train[:, 0],y_train)
plt.scatter(X_train[:, 1],y_train)

In [ ]:
# Plot feature 3: Proportion of non-retail business acres per town
plt.scatter(X_train[:, 2],y_train)

In [ ]:
# Plot feature 4: Charles River dummy variable (= 1 if tract bounds river, 0 otherwise)
plt.scatter(X_train[:, 3],y_train)

In [ ]:
# Plot feature 5: Nitric oxides concentration (parts per 10 million)
plt.scatter(X_train[:, 4],y_train)

In [ ]:
# Plot feature 6: Average number fo rooms per dwelling
plt.scatter(X_train[:, 5],y_train)

In [ ]:
# Plot feature 7: Porportion of owner-occupied units built prior to 1940
plt.scatter(X_train[:, 6],y_train)

In [ ]:
# Plot feature 8: Weighted distances to five Boston employment centers
plt.scatter(X_train[:, 7],y_train)

In [ ]:
# Plot feature 9: Index of accessability to radial highways
plt.scatter(X_train[:, 8],y_train)

In [ ]:
# Plot feature 10: Full-value property-tax rate per $10,000
plt.scatter(X_train[:, 9],y_train)

In [ ]:
# Plot feature 11: Pupil-teacher ratio by town
plt.scatter(X_train[:, 10],y_train)

In [ ]:
# Plot feature 12: 1000(Bk - 0.63)^2 where Bk is the population fo blacks by town
plt.scatter(X_train[:, 11],y_train)

In [ ]:
# Plot feature 13: % lower status of the population
plt.scatter(X_train[:, 12],y_train)